In [1]:
# Module Imports
import boto3
import cv2
import os
import uuid
import threading
from tkinter import Tk, Button, Label, StringVar, filedialog, messagebox


In [3]:
#  AWS Configuration

S3_BUCKET = 'imageswithboto3'
SNS_TOPIC_ARN = 'arn:aws:sns:ap-south-1:686255986807:s3upload'

# Initialize AWS Clients
s3 = boto3.client('s3')
sns = boto3.client('sns')


In [5]:
# GUI Setup

root = Tk()
root.title("📤 Smart S3 Image Uploader")
root.geometry("600x420")
root.resizable(False, False)
root.configure(bg="#f4f4f4")

status_text = StringVar()
status_text.set("🟢 Ready")


# 📁 Use Case 1: Upload Image from user


def select_from_file():
    file_path = filedialog.askopenfilename(
        title="Select an image to upload",
        filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp *.gif")]
    )
    if not file_path:
        status_text.set("❌ No file selected.")
        return
    filename = os.path.basename(file_path)
    preview_and_upload(file_path, filename)

In [7]:
#  Use Case 2: Capture Image from Webcam

def capture_from_camera():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Camera Error", "Unable to access webcam.")
        return

    status_text.set("📸 Capturing... Press SPACE to snap")

    while True:
        ret, frame = cap.read()
        if not ret:
            status_text.set("❌ Camera read failed")
            break
        cv2.imshow("Press SPACE to Capture | ESC to Cancel", frame)

        key = cv2.waitKey(1)
        if key == 27:  # ESC
            cap.release()
            cv2.destroyAllWindows()
            status_text.set("❌ Capture cancelled")
            return
        elif key == 32:  # SPACE
            filename = f"captured_{uuid.uuid4().hex[:8]}.jpg"
            filepath = os.path.join(os.getcwd(), filename)
            cv2.imwrite(filepath, frame)
            cap.release()
            cv2.destroyAllWindows()
            preview_and_upload(filepath, filename)
            return



In [9]:
#  Use Case 3: Preview Image with OpenCV

def preview_and_upload(image_path, filename):
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise Exception("Unable to read image.")
        cv2.imshow("Preview - Press any key to upload", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        threading.Thread(target=upload_to_s3_and_notify, args=(image_path, filename)).start()
    except Exception as e:
        status_text.set(f"❌ OpenCV error: {e}")
        messagebox.showerror("Error", str(e))


#  Use Case 4: Upload to S3 & Notify via SNS

def upload_to_s3_and_notify(image_path, filename):
    try:
        # Upload to S3
        s3.upload_file(image_path, S3_BUCKET, filename)
        status_text.set(f"✅ Uploaded: {filename}")
         # Send SNS Notification
        message = f"🖼️ Image '{filename}' has been uploaded to your bucket '{S3_BUCKET}'."
        sns.publish(
            TopicArn=SNS_TOPIC_ARN,
            Subject="S3 Upload Notification",
            Message=message
        )

        messagebox.showinfo("Success", "Upload complete and email sent.")
    except Exception as e:
        status_text.set("❌ Upload failed")
        messagebox.showerror("Upload Error", str(e))



In [ ]:
 # GUI Widgets

Label(root, text="📷 Upload Image to AWS S3", font=("Helvetica", 18, "bold"), bg="#f4f4f4").pack(pady=30)

btn_file = Button(root, text="📁 Upload from File", command=select_from_file,
                  font=('Arial', 13), bg="#3498db", fg="white", width=30, height=2)
btn_file.pack(pady=15)

btn_camera = Button(root, text="📸 Capture from Webcam", command=capture_from_camera,
                    font=('Arial', 13), bg="#2ecc71", fg="white", width=30, height=2)
btn_camera.pack(pady=15)

Label(root, textvariable=status_text, font=("Arial", 11), fg="black", bg="#f4f4f4").pack(pady=20)

Label(root, text="Built with Python | Boto3 | OpenCV | Tkinter", font=("Arial", 9), fg="gray", bg="#f4f4f4").pack(side="bottom", pady=15)


# Launch GUI
root.mainloop()
